In [ ]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [ ]:
# model_name = "/models/models--microsoft--phi-2/snapshots/85d00b03fee509307549d823fdd095473ba5197c"
# model_name = "/models/models--TomGrc--FusionNet_SOLAR/snapshots/3757984c0edebf4300a67cf33b9cca53524a057d"
model_name = "/models/models--NousResearch--Llama-2-13b-chat-hf/snapshots/aad3c0b2315a458d54cd9b09ddc51cf06021196f"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    load_in_8bit_fp32_cpu_offload=True,
)

if False:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cuda",
        trust_remote_code=True
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        ),
        device_map="auto",
        trust_remote_code=True
    )
    

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
inputs = tokenizer('''Instruction: Turn the block of text below into a series of question/answer pairs, preserving as much of the text as possible.  The questions should elecit questions whose answers are quotations from the text.  The answers should not contain quotation marks.

"""
{text}
"""

Question: ''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(inputs.get("input_ids").to(model.device), max_length=1024)
text = tokenizer.batch_decode(outputs)[0]
print(text)